## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [4]:
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [6]:
arqEnem = r'E:\Estudos\SQL\Datasets\Enem\2022\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'

dfDadosGerais = spark.read.csv(arqEnem, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')


In [7]:
dfProvasObjetiva = dfDadosGerais.select(dfDadosGerais.columns[23:44])

In [8]:
dfProvasObjetiva.show()

+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|CO_PROVA_CN|CO_PROVA_CH|CO_PROVA_LC|CO_PROVA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|TP_LINGUA|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|
+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|   

In [9]:
dfPreencher = dfProvasObjetiva.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [10]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "ProvasObjetivas"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [11]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [12]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [13]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [ ]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')
